## data
### for grabbing / processing data

In [5]:
from evaltools.data import *
import pandas as pd
import us

### census
Uses the US Census Bureau's API to retrieve 2020 Decennnial Census PL 94-171 data at the stated geometry level. The five tables are
 * P1: Race
 * P2: Hispanic or Latino, and Not Hispanic or Latino by Race
 * P3: Race for the Population 18 Years and Over (Race by VAP)
 * P4: Hispanic or Latino, and Not Hispanic or Latino by Race for the Population 18 Years and Over
 * P5: Group Quarters Population by Major Group Quarters Type

In [92]:
%%time
df = census(us.states.MA, 
            table="P1", # Table from which we retrieve data, defaults to "P1"
            columns={}, # mapping Census column names from the table to human-readable names, if desired
            geometry="block group", # data granularity, one of "block" (default), "block group", or "tract"
           )

CPU times: user 147 ms, sys: 28.3 ms, total: 175 ms
Wall time: 3.17 s


In [63]:
mapping = variables("P4")

### acs5
Uses the US Census Bureau's API to retrieve 5-year population estimates from the American Community Survey (ACS) for the provided state, geometry level, and year.

In [50]:
%%time
df = acs5(us.states.MA,
          geometry="tract", # data granularity, either "tract" (default) or "block group"
          year=2019, # Year for which data is retrieved. Defaults to 2019, i.e. 2015-19 ACS 5-year
         )

CPU times: user 523 ms, sys: 44.7 ms, total: 567 ms
Wall time: 11.6 s


### cvap
Uses the US Census Bureau's API to retrieve the 2019 5-year CVAP (Citizen Voting Age Population) data for the provided state at the specified geometry. Please note that the geometries are from the **2010 Census**.

In [59]:
%%time
df = cvap(us.states.MA,
          geometry="tract", # data granularity, either "tract" (default) or "block group"
         )

CPU times: user 3.08 s, sys: 227 ms, total: 3.31 s
Wall time: 3.32 s
